In [ ]:
import numpy as np
import pandas as pd
import spacy
import nltk
import string
import torch
import torch.nn as nn

### Read the GloVe embeddings

In [ ]:
%run Kaggle_CommonLit_Challenge_Func.ipynb

In [ ]:
# Read the Glove embeddings for each word token
embeddings_dict = {}

with open('/glove_6B/glove.6B.50d.txt', 'r', encoding="utf-8") as f:
    for line in f:
        values = line.split()
        word = values[0]
        vector = np.asarray(values[1:], "float32")
        embeddings_dict[word] = vector

In [ ]:
# Number of word tokens
len(embeddings_dict)

In [ ]:
keyset = list(embeddings_dict)
# keyset

### Identify stopwords

In [ ]:
punctuation_labels = list(string.punctuation)
# punctuation_labels

In [ ]:
nlp = spacy.load("en_core_web_sm")

In [ ]:
spacy_stopwords = spacy.lang.en.stop_words.STOP_WORDS

In [ ]:
spacy_stopwords = spacy_stopwords | set(punctuation_labels)
len(spacy_stopwords)

### Generate the word embedding matrix per excerpt

In [ ]:
trng_set = pd.read_csv('/Kaggle/CommonLit_Readability/train.csv')
trng_set.head()

In [ ]:
trng_set.shape

In [ ]:
trng_set['excerpt'][0]

### Step 1: Identify the common number of words across all excerpts

In [ ]:
non_missing_words_per_excerpt = []
num_excerpts = trng_set.shape[0]

for idx in range(num_excerpts):
    if idx%100 == 0:
        print(idx)
    non_missing_words_per_excerpt.append(excerpt_embedding(trng_set['excerpt'][idx], 0))

In [ ]:
pd.Series(non_missing_words_per_excerpt).describe()

### Step 2: Consider only those excerpts for which the word length exceeds 77

In [ ]:
idx_valid_excerpt = []
num_excerpts = trng_set.shape[0]
trng_embeddings = np.zeros((1,50))

for idx in range(num_excerpts):
    if idx%100 == 0:
        print(idx)    
    
    temp = excerpt_embedding(trng_set['excerpt'][idx], 1)
    
    if temp.shape[0] >= 77:
        idx_valid_excerpt.append(idx)
        trng_embeddings = np.concatenate((trng_embeddings, temp[0:77, :]), axis=0)

In [ ]:
trng_embeddings.shape

In [ ]:
165242/77

In [ ]:
len(idx_valid_excerpt)

In [ ]:
# Save the excerpt embeddings along with the output values
output_matrix = trng_set.iloc[idx_valid_excerpt, [4, 5]]
output_matrix.to_csv('Scores_Valid_Excerpt.csv', index=False)

In [ ]:
np.save('Embeddings_Valid_Excerpt.npy', trng_embeddings[1:, :])